In [ ]:
from sklearn.metrics import log_loss
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd

predict_data = pd.read_csv('../data/training/predict/predict_data.csv')

positive_data = pd.read_csv('../data/training/positive/positive_data.csv')
positive_data['label'] = 1

negative_data = pd.read_csv('../data/training/negative/negative_data.csv')
negative_data['label'] = 0

data = pd.concat([positive_data, negative_data], ignore_index=True)
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

#replace inf distance with -1
data['distance'] = data['distance'].replace([float('inf'), -float('inf')], -1)
predict_data['distance'] = predict_data['distance'].replace([float('inf'), -float('inf')], -1)

#use ratio of abstract lengths instead of absolute lengths
data['abs_len_ratio'] = data['source_abstract_length'] / data['target_abstract_length']
predict_data['abs_len_ratio'] = predict_data['source_abstract_length'] / predict_data['target_abstract_length']

# Replace infinite values and clip large values in abs_len_ratio
data['abs_len_ratio'] = data['abs_len_ratio'].replace([float('inf'), -float('inf')], -1)
data['abs_len_ratio'] = data['abs_len_ratio'].clip(lower=-1e10, upper=1e10)
data['abs_len_ratio'] = data['abs_len_ratio'].fillna(-1)  # Fill NaN values with -1

predict_data['abs_len_ratio'] = predict_data['abs_len_ratio'].replace([float('inf'), -float('inf')], -1)
predict_data['abs_len_ratio'] = predict_data['abs_len_ratio'].clip(lower=-1e10, upper=1e10)
predict_data['abs_len_ratio'] = predict_data['abs_len_ratio'].fillna(-1)  # Fill NaN values with -1

X = data[[
    'source_pagerank',
    'target_pagerank',
    'source_k_core',
    'target_k_core',
    'distance',
    'dot_product',
    'euclidean_distance',
    'cosine_similarity',
    'abs_len_ratio',
    'source_abstract_length',
    'target_abstract_length',
    'jaccard_score'
]].values

y = data['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define and train the MLP
mlp = MLPClassifier(hidden_layer_sizes=(5, 5, 5), solver='adam', activation='relu', warm_start=True)
mlp.fit(X_train, y_train)
y_prob = mlp.predict_proba(X_test)[:, 1]  # Probability of class 1 (citation)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Predicted probabilities:", y_prob)
print("Accuracy:", accuracy_score(y_test, mlp.predict(X_test)))
print("Precision:", precision_score(y_test, mlp.predict(X_test)))
print("Recall:", recall_score(y_test, mlp.predict(X_test)))
print("F1 Score:", f1_score(y_test, mlp.predict(X_test)))
print("Log Loss:", log_loss(y_test, y_prob))

X_predict = predict_data[[
    'source_pagerank',
    'target_pagerank',
    'source_k_core',
    'target_k_core',
    'distance',
    'dot_product',
    'euclidean_distance',
    'cosine_similarity',
    'abs_len_ratio',
    'source_abstract_length',
    'target_abstract_length',
    'jaccard_score'
]].values
X_predict = scaler.transform(X_predict)

res = mlp.predict_proba(X_predict)[:, 1]
resdf = pd.DataFrame(res, columns=['probability'])
resdf = resdf.reset_index()
resdf = resdf.rename(columns={'index': 'ID', 'probability': 'Label'})

resdf[['ID', 'Label']].to_csv('../predictions/mlp_predictions.csv', index=False)